In [ ]:
import cv2
import os
from PIL import Image
import numpy as np

***
![](icons/inputs.png)
***
# <center>Inputs</center>
***

**model**: *An image of the target at which we shoot. Should be as clean as possible and must be arrows-free.*
<br>
**video_name**: *The path of the video we analyze.*
<br>
**bullseye_point**: *An (x,y) coordinates tuple of the bull'seye location in the target image.*
<br>
**inner_diameter_px**: *The diameter in pixels of the most inner ring in the target image (the faded ring inside the 10 ring).*
<br>
**inner_diameter_in**: *The diameter in inches of the most inner ring in the target image (the faded ring inside the 10 ring).*
<br>
**rings_amount**: *Amount of rings in the target.*
<br>
**display_in_cm**: *True to display measures in centimeters instead of inches.*

In [ ]:
model = cv2.imread('target.jpg')
video_name = 'video.mp4'
bullseye_point = (325,309)
inner_diameter_px = 50
inner_diameter_inch = 1.5
rings_amount = 6
display_in_cm = False

***
![](icons/preperations.png)
***
# <center>Preperations</center>
***
Calculate everything we can in advance before the analysis starts.

In [ ]:
# get a sample frame from the video
cap = cv2.VideoCapture(video_name)
_, test_sample = cap.read()

# calculate the sizes of the frame and the input
model_h, model_w, _ = model.shape
frame_h, frame_w, _ = test_sample.shape
pixel_to_inch = inner_diameter_inch / inner_diameter_px
pixel_to_cm = pixel_to_inch * 2.54
measure_unit = pixel_to_cm if display_in_cm else pixel_to_inch
measure_unit_name = 'cm' if display_in_cm else '"'

In [ ]:
# create a zero padded image and get its anchor points
def _zero_pad_as(img, padding_shape):
    img_h, img_w, _ = img.shape
    p_h, p_w, _ = padding_shape
    vertical = int((p_h - img_h) / 2)
    horizontal = int((p_w - img_w) / 2)
    a = (horizontal,vertical)
    b = (horizontal + img_w,vertical)
    c = (horizontal + img_w,vertical + img_h)
    d = (horizontal,vertical + img_h)
    e = (int(horizontal + img_w / 2),int(vertical + img_h / 2))
    pad_img = cv2.copyMakeBorder(img, vertical, vertical, horizontal, horizontal, cv2.BORDER_CONSTANT)
    anchor_points = [a, b, c, d, e]

    return anchor_points, pad_img

anchor_points, pad_model = _zero_pad_as(model, test_sample.shape)
anchor_a = anchor_points[0]
bullseye_anchor = (anchor_a[0] + bullseye_point[0],anchor_a[1] + bullseye_point[1])
anchor_points.append(bullseye_anchor)
anchor_points = np.float32(anchor_points).reshape(-1, 1, 2)

In [ ]:
sift = cv2.xfeatures2d.SIFT_create()
model_keys, model_desc = sift.detectAndCompute(pad_model, None)

***
![](icons/matching.png)
***
# <center>Feature Matching</center>
***

Take the model image and find it in the frame.

In [ ]:
def ratio_match(matcher, query_desc, train, ratio):
    train_keys, train_desc = matcher.detectAndCompute(train, None)
    bf = cv2.BFMatcher(crossCheck=False)
    best_match = []
    
    if type(train_desc) != type(None):
        # apply ratio test
        matches = bf.knnMatch(query_desc, train_desc, k=2)

        try:
            for m1, m2 in matches:
                if m1.distance < ratio * m2.distance:
                    best_match.append(m1)
        except ValueError:
            return [], ([], [])

    return best_match, (train_keys, train_desc)

![](icons/homography.png)
***
# <center>Homography</center>
***

Calculate the homography of the target image that had been found in the frame.

In [ ]:
def calc_homography(query_keys, train_keys, matches):
    if not len(matches):
        return None

    # reshape keypoints
    src_pts = np.float32([query_keys[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([train_keys[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
    
    H, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5)
    return H

Check if the homography is realistic enough, and not TOO stretched.<br>
This function checks the ratio between the homography edges,
and that the vertices are in the expected 2D order.

In [ ]:
def is_true_homography(vertices, edges, img_size, stretch_threshold):
    A = vertices[0]
    B = vertices[1]
    C = vertices[2]
    D = vertices[3]
    E = vertices[4]
    upsidedown = B[0] < A[0]

    if upsidedown:
        c_ordered = C[0] < D[0] and C[1] < B[1]
        d_ordered = D[1] < A[1]
        e_ordered = E[0] < D[0] and E[0] > B[0]
    else:
        c_ordered = C[0] > D[0] and C[1] > B[1]
        d_ordered = D[1] > A[1]
        e_ordered = E[0] > D[0] and E[0] < B[0]

    ab = edges[0]
    bc = edges[1]
    cd = edges[2]
    da = edges[3]

    unstretched_hor = ab / cd >= 1 - stretch_threshold and ab / cd <= 1 + stretch_threshold
    unstretched_ver = bc / da >= 1 - stretch_threshold and bc / da <= 1 + stretch_threshold

    unstretched = unstretched_hor and unstretched_ver
    all_ordered = c_ordered and d_ordered and e_ordered
    vals_arr = np.array([A[0],A[1],B[0],B[1],C[0],C[1],D[0],D[1],E[0],E[1]])
    out_of_bounds = (vals_arr < 0).any() or (vals_arr > max(img_size[0], img_size[1])).any()

    return unstretched and all_ordered and not out_of_bounds

***
![](icons/2d.png)
***
# <center>2D Geometry Utility Calculations</center>
***

A simple eucliedean distance formula that we will be using now and then.

In [ ]:
def euclidean_dist(p1, p2):
    return ((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2) ** .5

Take the model image and the detected object in the frame (which are hopefully the same object), and calculate these 3 attributes:

* **horizontal percentage**: *Horizontal scale ratio of the model's edges divided by the object's edges.*
<br>
* **vertical percentage**: *Vertical scale ratio of the model's edges divided by the object's edges.*
<br>
* **scale percentage**: *Average of both the horizontal and vertical scale ratios. It sheds more light on their ratio difference.*

In [ ]:
def calc_model_scale(edges, model_shape):
    horizontal_edge = (edges[0] + edges[2]) / 2
    vertical_edge = (edges[1] + edges[3]) / 2
    hor_percent = horizontal_edge / vertical_edge
    ver_percent = vertical_edge / horizontal_edge
    hor_scale = horizontal_edge / model_shape[1]
    ver_scale = vertical_edge / model_shape[0]
    scale_percent = (hor_scale + ver_scale) / 2

    return hor_percent, ver_percent, scale_percent

Calculate the 4 coordinates and 4 edge lengths of a given rectangular transformation.

In [ ]:
def calc_vertices_and_edges(transform):
    vertices = [transform[m][0] for m in range(len(transform))]
    A = vertices[0]
    B = vertices[1]
    C = vertices[2]
    D = vertices[3]
    ab = (((A[0] - B[0]) ** 2) + ((A[1] - B[1]) ** 2)) ** .5
    bc = (((B[0] - C[0]) ** 2) + ((B[1] - C[1]) ** 2)) ** .5
    cd = (((C[0] - D[0]) ** 2) + ((C[1] - D[1]) ** 2)) ** .5
    da = (((D[0] - A[0]) ** 2) + ((D[1] - A[1]) ** 2)) ** .5

    return vertices, (ab, bc, cd, da)

Calculate the euclidean distance of each pixel in the image from the target's bull'seye point.

In [ ]:
def calc_bullseye_pixels_distances(frame_size, bullseye):
    dx = np.arange(frame_size[1])
    dy = np.arange(frame_size[0])
    x, y = bullseye[0], bullseye[1]
    mat_X, mat_Y = np.meshgrid(dx, dy)
    distances = ((mat_X, mat_Y), euclidean_dist((mat_X,mat_Y), (x,y)))
    return distances

***
![](icons/contours.png)
***
# <center>Contours Classification</center>
***

Find the distance of each pixel in a contour from a specified point, and return a sorted list.

In [ ]:
def contour_distances_from(contourPts, point):
    pts = [[p[0], p[1], 0] for p in contourPts]
    
    for i in range(len(pts)):
        p = pts[i]
        xy = (p[0],p[1])
        p[2] = euclidean_dist(xy, point)
    
    return sorted(pts, key=lambda x: x[2])

Extend the straight contour line owtwards the target, to try and reproduce the shape and length of the actual projectile.<br>
This helps joining multiple contours in a row, that refer to the same projectile.

In [ ]:
def extend_contour_line(img, contour, bullseye, length):
    def normalize(vector):
        square_sum = 0
        for x in vector:
            square_sum += x ** 2

        magnitude = square_sum ** .5
        return np.array(vector) / magnitude

    # find a rectangle that strictly bounds the contour
    bounding_rect = cv2.minAreaRect(contour)
    box = cv2.boxPoints(bounding_rect)
    box = np.int0(box)
    A = box[0]
    B = box[1]
    C = box[2]
    D = box[3]

    # find the two shorter edges
    AB = euclidean_dist(A, B)
    BC = euclidean_dist(B, C)

    if AB < BC:
        edge_1_pts = (A,B)
        edge_2_pts = (C,D)
    else:
        edge_1_pts = (B,C)
        edge_2_pts = (A,D)

    # calculate the middle points of the two edges
    alpha = (int((edge_1_pts[0][0] + edge_1_pts[1][0]) / 2),int((edge_1_pts[0][1] + edge_1_pts[1][1]) / 2))
    beta = (int((edge_2_pts[0][0] + edge_2_pts[1][0]) / 2),int((edge_2_pts[0][1] + edge_2_pts[1][1]) / 2))

    # decide which edge is closer to the target's bulls'eye point
    alpha_dist = euclidean_dist(alpha, bullseye)
    beta_dist = euclidean_dist(beta, bullseye)
    front_point = alpha if alpha_dist < beta_dist else beta
    rear_point = beta if alpha_dist < beta_dist else alpha

    # calculate the estimated point of the projectile's back
    direction = normalize(np.array(rear_point) - np.array(front_point))
    end_point = np.array(front_point) + direction * length
    end_point = end_point.tolist()
    end_point = tuple([int(x) for x in end_point])

    cv2.line(img, front_point, end_point, (0xff,0x0,0xff), 4)

Check if a contour is a rectangular or is it convexed (moon-shaped).<br>
This helps eliminating the outlier contours that are generated from the target's rings.

In [ ]:
def is_contour_rect(cont, A, B, samples):
    x_distance = B[0] - A[0]
    y_distance = B[1] - A[1]
    x_step = x_distance / samples
    y_step = y_distance / samples

    # contour is a very small square
    if (x_step == 0 or y_step == 0):
        return False

    x_vals = np.arange(A[0], B[0], x_step)
    y_vals = np.arange(A[1], B[1], y_step)
    points = [(x,y) for x, y in zip(x_vals, y_vals)]

    for p in points:
        # check if point is outside the contour
        if cv2.pointPolygonTest(cont, p, False) < 0:
            return False

    return True

Take a list of contours and filter out all the contours with a convex shape, which are defenitely not straight projectiles.

In [ ]:
def filter_convex_contours(contours):
    filtered = []

    for cont in contours:
        contPts = [(cont[m][0][0],cont[m][0][1]) for m in range(len(cont))]
        point_A = contPts[0] # some random point on the contour

        # find the two furthest points on the contour
        point_B = contour_distances_from(contPts, point_A)[::-1][0]
        point_A = contour_distances_from(contPts, point_B)[::-1][0]

        # calculate the point between the two
        point_C = ((point_A[0] + point_B[0]) / 2, (point_A[1] + point_B[1]) / 2)

        # if this point is outside the contour, it's convex,
        # if it's inside it, the contour is relatively straight
        if is_contour_rect(cont, point_A, point_B, 5):
            filtered.append(cont)

    return filtered

***
![](icons/compvis.png)
***
# <center>Visual Analysis</center>
***

Take both the model image and the detected object in the frame and perform a background subtraction,<br>
idealy creating in an image of the arrows alone.

In [ ]:
def subtract_background(query, subtrahend):
    # convert to grayscale
    gray_query = cv2.cvtColor(query, cv2.COLOR_RGB2GRAY)
    gray_subtrahend = cv2.cvtColor(subtrahend, cv2.COLOR_RGB2GRAY)

    # apply gaussian blur
    kernel = (3,3)
    gray_query = cv2.GaussianBlur(gray_query, kernel, 0)
    gray_subtrahend = cv2.GaussianBlur(gray_subtrahend, kernel, 0)

    # apply a black area on the subtrahend image
    gray_subtrahend[gray_query == 0] = 0

    # calculate diff
    diff = cv2.absdiff(gray_subtrahend, gray_query)
    return diff

Emphasize all of the straight lines in the image and get rid of unnecessary noise.

In [ ]:
def emphasize_lines(img, distances, estimated_radius):
    # find the target's outer ring
    circles = cv2.HoughCircles(img, cv2.HOUGH_GRADIENT, 1, 20,
                               param1=50, param2=30, minRadius=0,
                               maxRadius=int(estimated_radius * 1.05))
    
    # use largest detected circle
    if type(circles) != type(None):
        outerCircle = sorted(circles[0], key=lambda x: x[2])[::-1][0]
        radius = outerCircle[2]
        
    # use a rough estimation of the target's radius as a fallback
    else:
        radius = estimated_radius

    # zero out all pixels outside of the outer ring
    img[distances[1] > radius] = 0
    
    # apply thresh and morphology
    _, img = cv2.threshold(img, 20, 0xff, cv2.THRESH_BINARY)
    img = cv2.morphologyEx(img, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))

    # find the straight segments in the image
    lines = cv2.HoughLinesP(img, 2, np.pi / 180, 120, minLineLength=20, maxLineGap=0)
    img_copy = np.zeros(img.shape, dtype=img.dtype)

    if type(lines) != type(None):
        for line in lines:
            for x1, y1, x2, y2 in line:
                cv2.line(img_copy, (x1, y1), (x2, y2), (0xff,0xff,0xff), 5)
                
    return radius, img_copy

Extend the emphasized lines outwards the target circle in order to restore the shape of the projectiles that might have been broken during the process.

In [ ]:
def reproduce_projectile_contours(img, distances, bullseye, radius):
    # detect the unconvex contours (true projectile contours)
    contours = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[-2:]
    rect_contours = filter_convex_contours(contours[0])
    blank_img = np.zeros(img.shape, dtype=img.dtype)
    
    for cont in rect_contours:
        extend_contour_line(blank_img, cont, bullseye, length=radius)
    
    # clear unnecessary noise
    blank_img[distances[1] > radius] = 0
    blank_img = cv2.morphologyEx(blank_img, cv2.MORPH_CLOSE, np.ones((3,3), np.uint8))
    
    # detect contours again, after the extension
    return cv2.findContours(blank_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[-2:][0]

Assume all the contours that are left are indeed projectiles, and calculate the distances of their pointy edges from the target's bull'seye point.

In [ ]:
def find_suspect_hits(contours, vertices, scale):
    bullseye = vertices[5]
    res = []
    
    for cont in contours:
        contPts = [(cont[m][0][0],cont[m][0][1]) for m in range(len(cont))]
        point_A = contPts[0] # some random point on the contour

        # find the two furthest points on the contour
        point_B = contour_distances_from(contPts, point_A)[::-1][0]
        point_A = contour_distances_from(contPts, point_B)[::-1][0]
        
        # decide which of them is closer to the bullseye point
        A_dist = euclidean_dist(point_A, bullseye)
        B_dist = euclidean_dist(point_B, bullseye)
        hit = point_A if A_dist < B_dist else point_B

        # straighten the target's oval and find the real hit values
        res_x = (hit[0] - vertices[0][0]) * scale[0] + vertices[0][0]
        res_y = (hit[1] - vertices[0][1]) * scale[1] + vertices[0][1]
        res_dist = euclidean_dist(hit, bullseye)
        res_hit = (res_x,res_y,res_dist, bullseye)
        res.append(res_hit)

    return res

***
![](icons/detection.png)
***
# <center>Score Detection System</center>
***

It is necessary to design a customized class for detected hits.

**point**: *The (x,y) coordinates of the hit.*
<br>
**score**: *The calculated score of the hit according to the rules.*
<br>
**reputation**: *The consistency of the hit over the frames.<br>
A hit with high reputation means it has been detected over and over again over multiple consistent frames.*
<br>
**bullseye_relation**: *The bull'seye coordinates of the target from the last time the hit had been detected.*

In [ ]:
class Hit:
    def __init__(self, x, y, score, bullseyeRelation):
        self.point = (x, y)
        self.score = score
        self.reputation = 1
        self.bullseye_relation = bullseyeRelation
        
        # has this hit been checked during current iteration
        self.iter_mark = False
    
    def increase_rep(self):
        self.reputation += 1
        
    def decrease_rep(self):
        self.reputation -= 1
        
    def isVerified(self, repScore):
        return self.reputation >= repScore

## Score calculation
![10 rings target](images/fita_target.png)

In [ ]:
def calc_score(hits, scale):
    scoreboard = []
    
    for hit in hits:
        hit_dist = hit[2]
        scaled_diam = inner_diameter_px * scale[2]
        score = 10 - int(hit_dist / scaled_diam)

        # clamp score between 10 and minimum available score
        if score < 10 - rings_amount + 1:
            score = 0
        elif score > 10:
            score = 10
        
        hit_obj = Hit(int(hit[0]), int(hit[1]), score, hit[3])
        scoreboard.append(hit_obj)

    return scoreboard

## Score classification

Initially, we need 2 lists:
* A list for candidate hits, that are not yet confirmed to be real
* A list for verified confirmed hits

In [ ]:
candidate_hits = []
verified_hits = []

We need a way to search within those lists and check if each of the hits we detect is already known.<br>
**distanceTolerance** is a parameter that allows us to indentify a detected hit in the lists, even if it is no longer located at the exact coordinates as before.

In [ ]:
def is_verified_hit(point, distanceTolerance):
    return type(get_verified_hit(point, distanceTolerance)) != type(None)

def is_candidate_hit(point, distanceTolerance):
    return type(get_candidate_hit(point, distanceTolerance)) != type(None)

def get_verified_hit(point, distanceTolerance):
    compatible_hits = []
    
    for hit in verified_hits:
        if euclidean_dist(point, hit.point) <= distanceTolerance:
            compatible_hits.append(hit)
            
    if len(compatible_hits) > 0:
        return compatible_hits[0]
    else:
        return None;

def get_candidate_hit(point, distanceTolerance):
    compatible_hits = []
    
    for hit in candidate_hits:
        if euclidean_dist(point, hit.point) <= distanceTolerance:
            compatible_hits.append(hit)
            
    if len(compatible_hits) > 0:
        return compatible_hits[0]
    else:
        return None

Find duplicate verified hits and eliminate them.

In [ ]:
def eliminateVerifiedRedundancy(distanceTolerance):
    if len(verified_hits) <= 1:
        return
    
    # create a table of the distances between all hits
    table = np.ndarray((len(verified_hits),len(verified_hits)), np.float32)
    j_leap = 0
    for i in range(len(table)):
        for j in range(len(table[i])):
            col = j + j_leap
            if col >= len(table[i]):
                continue
            
            hit_i = verified_hits[i].point
            hit_j = verified_hits[col].point
            dist = euclidean_dist(hit_i, hit_j)
            table[i][col] = dist
            
        j_leap += 1
    
    # find distances that are smaller than the threshold and eliminate the redundant hits
    j_leap = 0
    for i in range(len(table)):
        for j in range(len(table[i])):
            col = j + j_leap
            if col >= len(verified_hits):
                continue
            
            if i == col or i >= len(verified_hits):
                continue
            
            if table[i][col] < distanceTolerance:
                hit_i = verified_hits[i].point
                hit_j = verified_hits[col].point
                
                # check the distance from the bull'seye point
                bullseye_i = verified_hits[i].bullseye_relation
                bullseye_j = verified_hits[col].bullseye_relation
                bullseye_dist_i = euclidean_dist(hit_i, bullseye_i)
                bullseye_dist_j = euclidean_dist(hit_j, bullseye_j)

                if bullseye_dist_i < bullseye_dist_j:
                    verified_hits.remove(verified_hits[col])
                else:
                    verified_hits.remove(verified_hits[i])
        
        j_leap += 1

### Sorting a newly detected hit

* <font color='green'><u>Already verified</u></font> - Don't change it.<br><br>

* <font color='red'><u>Already a candidate</u></font> - Increase its reputation.<br>
A candidate hit with enough reputation becomes verified.<br><br>

* <u>Unfamiliar</u> - Add it to the candidates list.

In [ ]:
def sort_hit(hit, distanceTolerance, maxReputation):
    candidate = get_candidate_hit(hit.point, distanceTolerance)

    # the hit is a known candidate
    if type(candidate) != type(None):
        candidate.increase_rep()
        candidate.iter_mark = True

        # candidate is now eligable for verification
        if candidate.isVerified(maxReputation):
            verified_hits.append(candidate)
            candidate_hits.remove(candidate)
            
            # find duplicate verified hits and eliminate them
            eliminateVerifiedRedundancy(distanceTolerance)

    # new candidate
    else:
        candidate_hits.append(hit)
        hit.iter_mark = True        

### Disqualifying candidate hits
The hits the are listed as candidates, but not detected during the last frame, have their reputation decreased.<br>
A reputation lower than 0 causes the removal of the hit from the list.

In [ ]:
def discharge_hits():
    for candidate in candidate_hits:
        # candidate is not present during the current iteration
        if not candidate.iter_mark:
            candidate.decrease_rep()
            
            # candidate disqualified
            if candidate.reputation <= 0:
                candidate_hits.remove(candidate)
                continue
        
        # get ready for the next iteration
        candidate.iter_mark = False

### Stabilizing hits during video movements
Shift each hit back to its correct location, relative to the bull'seye point in the current frame.

In [ ]:
def shift_hits(bullseye):
    all_hits = candidate_hits + verified_hits
    
    for h in all_hits:
        # find the correct translation amount
        x_dist = bullseye[0] - h.bullseye_relation[0]
        y_dist = bullseye[1] - h.bullseye_relation[1]
        new_x = int(round(h.point[0] + x_dist))
        new_y = int(round(h.point[1] + y_dist))
        
        # translate and update relation attribute
        h.bullseye_relation = bullseye
        h.point = (new_x,new_y)

***
![](icons/grouping.png)
***
# <center>Grouping Detection</center>
***

To find the group polygon we first need to draw the lines between all verified hits.<br>Then we take the contour of the shape that has been created.

In [ ]:
def create_group_polygon(img, hits):
    blank_img = np.zeros(img.shape, dtype=img.dtype)
    blank_img = cv2.cvtColor(blank_img, cv2.COLOR_RGB2GRAY)
    
    # draw lines between all hits
    for h1 in hits:
        for h2 in hits:
            if h1 != h2:
                cv2.line(blank_img, h1.point, h2.point, (0xff,0xff,0xff), 3)
    
    # find external contour
    contours = cv2.findContours(blank_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)[-2:]
    blank_img = cv2.cvtColor(blank_img, cv2.COLOR_GRAY2RGB)
    
    if len(contours[0]) > 0:
        return contours[0][0]
    else:
        return None

A group of arrows is measured by the distance of the two points that are furthest apart.<br>
The lower this distance is, the more valuable is the grouping.

In [ ]:
def measure_grouping_diameter(contour):
    # find two furthest points in the polygon
    contPts = [(contour[m][0][0],contour[m][0][1]) for m in range(len(contour))]
    point_A = contPts[0] # random point
    point_B = contour_distances_from(contPts, point_A)[::-1][0]
    point_A = contour_distances_from(contPts, point_B)[::-1][0]
    
    # find their distance for each other
    point_A = (point_A[0], point_A[1])
    point_B = (point_B[0], point_B[1])
    return euclidean_dist(point_A, point_B)

![](icons/mark.png)
***
# <center>Data Drawing</center>
***

In [ ]:
def mark_hits(img, hits, foreground, diam, withOutline, withScore):
    outline = (0x0,0x0,0x0)
    
    for hit in hits:
        x, y = hit.point[0], hit.point[1]
        score_string = str(hit.score) if (hit.score > 0) else 'miss'
        
        if withOutline:
            cv2.circle(img, (x,y), 13, outline, diam + 2)
            
        cv2.circle(img, (x,y), 10, foreground, diam)
        
        if withScore:
            cv2.putText(img, score_string, (x,y - 20), cv2.FONT_HERSHEY_PLAIN, 5, outline, thickness=15)
            cv2.putText(img, score_string, (x,y - 20), cv2.FONT_HERSHEY_PLAIN, 5, (0xff,0xff,0xff), thickness=5)

In [ ]:
def draw_grouping(img, contour):
    cv2.drawContours(img, contour, -1, (214,215,97), 2)

Draw the rectangle on which we display the overall data.

In [ ]:
def draw_meta_data_block(img):
    img_h, img_w, _ = img.shape
    
    rect_0_start = (int(img_w * .5), int(img_h * .85))
    rect_0_end = (img_w, img_h)
    cv2.rectangle(img, rect_0_start, rect_0_end, (0xff,0xff,0xff), -1)
    
    rect_1_start = (rect_0_start[0] - 60, int(img_h * .85))
    rect_1_end = (rect_0_start[0] - 15, img_h)
    cv2.rectangle(img, rect_1_start, rect_1_end, (0x28,0x28,0x28), -1)
    
    rect_2_start = (rect_1_start[0] - 50, int(img_h * .85))
    rect_2_end = (rect_1_start[0] - 15, img_h)
    cv2.rectangle(img, rect_2_start, rect_2_end, (248,138,8), -1)
    
    rect_3_start = (rect_2_start[0] - 40, int(img_h * .85))
    rect_3_end = (rect_2_start[0] - 15, img_h)
    cv2.rectangle(img, rect_3_start, rect_3_end, (66,0x0,0xff), -1)
    
    rect_4_start = (rect_3_start[0] - 30, int(img_h * .85))
    rect_4_end = (rect_3_start[0] - 15, img_h)
    cv2.rectangle(img, rect_4_start, rect_4_end, (0x0,204,0xff), -1)
    
    rect_5_start = (rect_4_start[0] - 20, int(img_h * .85))
    rect_5_end = (rect_4_start[0] - 15, img_h)
    cv2.rectangle(img, rect_5_start, rect_5_end, (0x0,204,0xff), -1)

In [ ]:
def type_arrows_amount(img, amount, dataColor):
    amount = str(amount)
    img_h, img_w, _ = img.shape
    cv2.putText(img, 'Arrows shot: ', (int(img_w * .52), int(img_h * .905)),
                cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0x0,0x0,0x0), 4)
    
    cv2.putText(img, amount, (int(img_w * .675), int(img_h * .905)),
                cv2.FONT_HERSHEY_SIMPLEX, 1.4, dataColor, 4)

In [ ]:
def type_grouping_diameter(img, diameter, dataColor):
    diameter = str(round(diameter * measure_unit, 1))
    img_h, img_w, _ = img.shape
    cv2.putText(img, 'Grouping: ', (int(img_w * .77), int(img_h * .905)),
                cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0x0,0x0,0x0), thickness=4)
    
    cv2.putText(img, diameter + measure_unit_name, (int(img_w * .89), int(img_h * .905)),
                cv2.FONT_HERSHEY_SIMPLEX, 1.4, dataColor, 4)

In [ ]:
def type_total_score(img, totalScore, achievableScore, dataColor):
    totalScore = str(totalScore)
    achievableScore = str(achievableScore)
    score_digits = len(totalScore)
    score_space = 23 * (score_digits - 1)
    img_h, img_w, _ = img.shape
    
    cv2.putText(img, 'Total score: ', (int(img_w * .52), int(img_h * .975)),
                cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0x0,0x0,0x0), thickness=4)
    
    cv2.putText(img, totalScore, (int(img_w * .67), int(img_h * .975)),
                cv2.FONT_HERSHEY_SIMPLEX, 1.4, dataColor, 4)
    
    cv2.putText(img, '/ ' + achievableScore, (int(img_w * .695 + score_space), int(img_h * .975)),
                cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0x0,0x0,0x0), 4)

![](icons/film.png)
***
# <center>Video Analysis</center>
***

In [ ]:
def analyze(frame):
    # set default analysis meta-data
    scoreboard = []
    scores = []
    bullseye_point = None

    # find a match between the model image and the frame
    matches, (train_keys, train_desc) = ratio_match(sift, model_desc, frame, .7)

    # start calculating homography
    if len(matches) >= 4:
        homography = calc_homography(model_keys, train_keys, matches)

        # check if homography succeeded and start warping the model over the detected object
        if type(homography) != type(None):
            warped_transform = cv2.perspectiveTransform(anchor_points, homography)
            warped_vertices, warped_edges = calc_vertices_and_edges(warped_transform)
            bullseye_point = warped_vertices[5]

            # check if homography is good enough to continue
            if is_true_homography(warped_vertices, warped_edges, (frame_w, frame_h), .2):
                # warp the input image over the filmed object and calculate the scale difference
                warped_img = cv2.warpPerspective(pad_model, homography, (frame_w, frame_h))
                scale = calc_model_scale(warped_edges, model.shape)
                
                # process image
                sub_target = subtract_background(warped_img, frame)
                pixel_distances = calc_bullseye_pixels_distances(frame.shape, warped_vertices[5])
                estimated_warped_radius = rings_amount * inner_diameter_px * scale[2]
                circle_radius, emphasized_lines = emphasize_lines(sub_target, pixel_distances,
                                                                  estimated_warped_radius)
                
                proj_contours = reproduce_projectile_contours(emphasized_lines, pixel_distances,
                                                              warped_vertices[5], circle_radius)
                
                suspect_hits = find_suspect_hits(proj_contours, warped_vertices, scale)

                # calculate hits and draw circles around them
                scoreboard = calc_score(suspect_hits, scale)

    return bullseye_point, scoreboard

In [ ]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v') # DIVX, XVID, MJPG, X264, WMV1, WMV2,...
out = cv2.VideoWriter('output.mp4',fourcc, 24.0, (frame_w,frame_h))

while True:
    ret, frame = cap.read()

    if ret:
        bullseye, scoreboard = analyze(frame)
        
        # increase reputation of consistent hits
        # or add them as new candidates
        for hit in scoreboard:
            sort_hit(hit, 30, 15)
        
        # decrease reputation of inconsistent hits
        discharge_hits()
        
        # stabilize all hits according to the slightly shifted bull'seye point
        if type(bullseye) != type(None):
            shift_hits(bullseye)
            
        # extract grouping data
        grouping_contour = create_group_polygon(frame, verified_hits)
        has_group = type(grouping_contour) != type(None)
        grouping_diameter = measure_grouping_diameter(grouping_contour) if has_group else 0
            
        # write meta data on frame
        draw_meta_data_block(frame)
        verified_scores = [h.score for h in verified_hits]
        arrows_amount = len(verified_scores)
        type_arrows_amount(frame, arrows_amount, (0x0,0x0,0xff))
        type_total_score(frame, sum(verified_scores), arrows_amount * 10, (0x0,189,62))
        type_grouping_diameter(frame, grouping_diameter, (0xff,133,14))
        
        
        # mark hits and grouping
        draw_grouping(frame, grouping_contour)
        mark_hits(frame, candidate_hits, foreground=(0x0,0x0,0xff),
                  diam=2, withOutline=False, withScore=False)
        
        mark_hits(frame, verified_hits, foreground=(0x0,0xff,0x0),
                  diam=5, withOutline=True, withScore=True)
        
        # display
        frame_resized = cv2.resize(frame, (1153, 648))
        cv2.imshow('Analysis', frame_resized)
        
        # write frame to output file
        out.write(frame)
        
        if cv2.waitKey(1) & 0xff == 27:
            break
    else:
        print('Video stream is over.')
        break
        
# close window properly
cap.release()
out.release()
cv2.destroyAllWindows()
cv2.waitKey(1)